In [1]:
#Import functions createdon my side
from Functions_custo_by_Nico import *
Test_Imported_Functions()


Functions have been properly imported !


In [2]:
src_path = os.path.abspath(os.path.join("../IA_Project10_Openclassrooms_Chatbot"))
if src_path not in sys.path:
    sys.path.append(src_path)

#Build Data path
data_path = os.path.join(src_path, "dataset")
print(data_path)
for f in os.listdir(src_path):
    print("|_",f)

c:\Users\blanc\OpenClassrooms\IA_Project10_Openclassrooms_Chatbot\dataset
|_ .git
|_ botbuilder
|_ dataset
|_ EDA.ipynb
|_ FlyMe_BOT_MVP
|_ Functions_custo_by_Nico.py
|_ Sources
|_ __pycache__


In [3]:
#Read data
data = pd.read_json(os.path.join(data_path, 'frames', 'frames.json'))
print("Dimensionnalité : ", data.shape)
data.head()

Dimensionnalité :  (1369, 5)


,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [4]:
data.describe(include='all')


,user_id,turns,wizard_id,id,labels
count,1369,1369,1369,1369,1369
unique,11,1369,12,1369,16
top,U22K1SX9N,[{'text': 'I'd like to book a trip to Atlantis...,U21T9NMKM,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
freq,345,1,301,1,929


#### Mise à plat des données

Comme nous avons pu le voir ci-dessous, les données Frames sont au format JSON.
CHaque dialogue BOT/User est constitué de **5 champs principaux: user_id, wizard_id, id, labels and turns**.
[Plus dinfo ici](https://www.microsoft.com/en-us/research/project/frames-dataset/download/)

Nous allons ci-dessous explorer les champs labels et turns, afin de comprendre leur structure respective

In [5]:
#Column Label : Segregate Json format and spread info in new dataframe columns
satisfaction_data = pd.json_normalize(data["labels"])
data["userSurveyRating"] = satisfaction_data["userSurveyRating"]
data["wizardSurveyTaskSuccessful"] = satisfaction_data["wizardSurveyTaskSuccessful"]
print("Il y a ", data[data["wizardSurveyTaskSuccessful"] == False].shape[0],
      "dialogues qui ont rendu un effet négatif.")
#data.describe(include='all')


Il y a  82 dialogues qui ont rendu un effet négatif.


In [6]:
# Cautious!!! Some values are not integer !!!
data["userSurveyRating"].unique()


array([4.  , 3.  , 2.  , 5.  , 1.  ,  nan, 4.8 , 4.5 , 4.99, 3.5 ])

In [28]:
#Column turns : Segregate Json format and spread info in new dataframe columns
entry_id = 0
data_turns = pd.DataFrame(data['turns'][entry_id])
data_turns

,text,labels,author,timestamp,db
0,I'd like to book a trip to Atlantis from Capri...,"{'acts': [{'args': [{'val': 'book', 'key': 'in...",user,1.471272e+12,NaN
1,"Hi...I checked a few options for you, and unfo...",{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471272e+12,{'result': [[{'trip': {'returning': {'duration...
2,"Yes, how about going to Neverland from Caprica...","{'acts': [{'args': [{'val': 'Neverland', 'key'...",user,1.471273e+12,NaN
3,I checked the availability for this date and t...,{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471273e+12,"{'result': [[], [], [], [], [], []], 'search':..."
4,I have no flexibility for dates... but I can l...,"{'acts': [{'args': [{'val': False, 'key': 'fle...",user,1.471273e+12,NaN
5,I checked the availability for that date and t...,{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471273e+12,"{'result': [[]], 'search': [{'ORIGIN_CITY': 'A..."
6,I suppose I'll speak with my husband to see if...,"{'acts': [{'args': [], 'name': 'thankyou'}], '...",user,1.471273e+12,NaN


In [42]:
data['turns'][entry_id][0]

{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'budget': [{'val': '1700.0', 'negated': False}],
     'dst_city': [{

In [8]:
turns_list = data['turns'].tolist()
turns_list[0][0].keys()


dict_keys(['text', 'labels', 'author', 'timestamp'])

In [54]:
turns_list[0][0]

{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'budget': [{'val': '1700.0', 'negated': False}],
     'dst_city': [{

In [9]:
#Study number of User/Wizard authoring text from dialogue
dic_author = {}
for turns in turns_list:
    for message in turns:
        author = message["author"]
        if not author in dic_author.keys():
            dic_author[author] = 1
        else:
            dic_author[author] += 1
dic_author


{'user': 10407, 'wizard': 9579}

In [52]:
# Extract values recognized by BOT and usefull for our application # In labels column: 'acts': [{'args': [{'val': 'book'
def get_key_value(arg):
    if "key" in arg and "val" in arg:
        if arg['val'] != "-1" and arg['val'] != None:
            key = arg["key"]
            val = arg["val"]
            return (key, val)
    return None

all_value = {}
for conversation in turns_list[:1]: #Nous allons chercher dans les 2 premiers echanges
    for message in conversation:
        # On récupére uniquement les echanges du user
        if message["author"] == "user":
            acts = message["labels"]["acts"]
            for act in acts:
                if act["args"]:
                    for arg in act["args"]:
                        key, value = get_key_value(arg)
                        if key in all_value:
                            all_value[key].append(value)
                        else:
                            all_value[key] = [value]
all_value


{'intent': ['book'],
 'dst_city': ['Atlantis', 'Neverland'],
 'or_city': ['Caprica', 'Atlantis'],
 'str_date': ['Saturday, August 13, 2016'],
 'n_adults': ['8', '5'],
 'budget': ['1700', '1900'],
 'ref': [[{'annotations': [{'val': 'Caprica', 'key': 'or_city'},
     {'val': 'August 13, 2016', 'key': 'str_date'}],
    'frame': 1,
    'fromrange': False}],
  [{'annotations': [{'val': 'Caprica', 'key': 'or_city'}],
    'frame': 1,
    'fromrange': False}]],
 'flex': [False]}

Comme expliqué [ici](https://learn.microsoft.com/fr-fr/azure/cognitive-services/luis/concepts/application-design), notre schéma d'application (LUIS) devra contenir les intentions et les entités pertinentes pour la réservation de voyage.


On remarque que notre jeu de données contient Frames.json les informations suivantes :
 - Intention: Reservation (**Book : spécifié dès le premier échange : *labels column: 'acts': [{'args': [{'val': 'book'***)
 - Destination: Atlantis ou Neverland
 - Origine: Caprica ou Atlantis
 - Date: 13 Aout 2016
 - nombre d'adultes: 8 ou 5
 - Budget: 1700 ou 1900
 - Ref: Nous retrouvons ici les Tuples imbriqués, qui font références au message antérieurs de l'utilisateur (Nous n'utiliserons pas cette information dans le cadre de notre projet V1)

Voici les informations clés suivantes à déterminer par notre BOT : 
- 'dst_city'
- 'or_city'
- 'str_date'
- 'end_date'
- 'n_adult'
- 'n_children'
- 'budget'
- 'seat'

### Convertir les informations de notre dataset pour le rendre compatible avec le format JSON attendu par LUIS

Nous devons concevoir le schéma d’application LUIS. notamment :
- Intentions avec des exemples d’énoncés (Book_flight)
- [Entités](https://learn.microsoft.com/fr-fr/azure/cognitive-services/luis/luis-concept-entity-types) (éléments de données extraits de l’énoncé, ex: destination, la date, la catégorie de ticket et la classe de voyage, le nombre de voyageurs)
- Fonctionnalités

Les entités LUIS permettent au BOT de comprendre les événements / intentions. Cela permet de collecter des informations supplémentaires, afin que le BOT puisse poser des questions et répondre plus intelligemment. 

le fichier FlightBooking.json d'origine contient les intentions : « Book Flight », « Cancel » et « None », avec un ensemble d’entités telles que « From.Airport » et « To.Airport ».
Nous modifierons ces information à partir de notre **jeu de données Frames.json**, et plus spécifiquement a agissant sur la partie **utterances**: 

In [11]:
#Here an example of Utterance content
# "utterances": [
#     {
#       "text": "book flight from london to paris on feb 14th",
#       "intent": "Book flight",
#       "entities": [
#         {
#           "entity": "To",
#           "startPos": 27,
#           "endPos": 31
#         },

In [12]:
def extract_entities(acts):
    dst_city = None
    or_city = None
    str_date = None
    end_date = None
    n_adults = None
    n_children = None #TBC
    budget = None
    seat = None #TBC
    intent = None
    for x in acts:
        z = x['args']
        if len(z) == 0:
            break
        for y in z:
            if 'dst_city' in y.values() and 'val' in y.keys():
                dst_city =  y['val']
            if 'or_city' in y.values() and 'val' in y.keys():
                or_city =  y['val']
            if 'str_date' in y.values() and 'val' in y.keys():
                str_date =  y['val']
            if 'end_date' in y.values() and 'val' in y.keys():
                end_date =  y['val']
            if 'n_adults' in y.values() and 'val' in y.keys():
                n_adults =  y['val']
            if 'n_children' in y.values() and 'val' in y.keys():
                n_children =  y['val']
            if 'budget' in y.values() and 'val' in y.keys():
                budget =  y['val']
            if 'seat' in y.values() and 'val' in y.keys():
                seat =  y['val']
            if 'intent' in y.values() and 'val' in y.keys():
                intent = y['val']
            
    return intent, or_city, dst_city, str_date, end_date, budget, n_adults, n_children, seat

In [171]:
def convert_to_luis_json(ids, json_labels={'text':'text', 'intent':'intent', 'entities':'entities', 'startPos':'startPos', 'endPos':'endPos', 'entity':'entity'}):
    json_top = []
    for i in ids:
        row = pd.DataFrame(data['turns'][i])
        # We will use only the first sentence of the dialog --> iloc[0]
        text = row.iloc[0]['text']
        acts = row.iloc[0]['labels']['acts']
        intent, or_city, dst_city, str_date, end_date, budget, n_adults, n_children, seat = extract_entities(acts)
        if intent == "book":  
            intent = "Book flight"
        elif intent == None:
            if any([or_city, dst_city, str_date, end_date, budget, n_adults, n_children, seat]):
                intent = "Book flight"
            else:
                continue
        entry_dict = {}
        entry_dict[json_labels['text']] = text
        entry_dict[json_labels['intent']] = intent
        entities = []
        #for k, v in zip(["From", "To"], [or_city, dst_city]) : #Working OK
        for k, v in zip(["From", "To", "budget", "n_adults", "n_children", "seat"], [or_city, dst_city, budget, n_adults, n_children, seat]) :
        #for k, v in zip(["From", "To", "str_date", "end_date", "budget", "n_adults", "n_children", "seat"], [or_city, dst_city, str_date, end_date, budget, n_adults, n_children, seat]) :
            if v is None or v == -1 or v == str(-1):
                continue
            try:
                v = re.escape(v)
                match=(re.search(v.lower(), text.lower()))
                entity = {json_labels["entity"]:k, json_labels["startPos"]:match.start(), json_labels["endPos"]:match.end() -1}
                entities.append(entity)
            except AttributeError:
                continue
        entry_dict[json_labels['entities']] = entities
        json_top.append(entry_dict)
    return json_top

In [172]:
#Let's try how the conversion to Json compatible with LUIS works
xxx = convert_to_luis_json(range(0,10))
print(json.dumps(xxx, indent=2))

[
  {
    "text": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
    "intent": "Book flight",
    "entities": [
      {
        "entity": "From",
        "startPos": 41,
        "endPos": 47
      },
      {
        "entity": "To",
        "startPos": 27,
        "endPos": 34
      },
      {
        "entity": "budget",
        "startPos": 117,
        "endPos": 120
      },
      {
        "entity": "n_adults",
        "startPos": 82,
        "endPos": 82
      }
    ]
  },
  {
    "text": "Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.",
    "intent": "Book flight",
    "entities": [
      {
        "entity": "From",
        "startPos": 44,
        "endPos": 54
      },
      {
        "entity": "To",
        "startPos": 59,
        "endPos": 68
      },
      {
        "entity": "budget",
        "startPos": 75,
        "endPos": 78
      }
    ]
  },
  {
    "text": "Hel

#### Conversion de l'ensemble du jeu de donnée et séparation en Train / Test

In [173]:
train_size = round(data.shape[0] * 0.9)
valid_size = data.shape[0] - train_size
train_json = convert_to_luis_json(range(0,train_size))
valid_json = convert_to_luis_json(range(train_size, train_size+valid_size))
with open('dataset/train.json', 'w') as file:
    json.dump(train_json, file)
    
with open('dataset/valid.json', 'w') as file:
    json.dump(valid_json, file)
    #json.dump(valid_json, file, indent=2)
len(train_json), len(valid_json)

(887, 134)

#### Ajout des données d'entrainement au fichier FlightBooking.json


In [170]:
#OK : starting from native FlightBooking file or template0
#with open('dataset/FlightBooking.json', 'r') as file:
with open('dataset/FlightBooking_template0.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified0.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

In [174]:
#KO : starting from template1 FlightBooking created manually
with open('dataset/FlightBooking_template1.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified1.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

### Autre exemple de conversion JSON

In [110]:
ls_entities = [
    "or_city",
    "dst_city",
    "str_date",
    "end_date",
    "budget"
]

def get_turn_entities(data, index, ls_entities):
    luis_data = []
    for conversation in data['turns'][index][:1]: # Nous n'utiliserons que les première entrée de l'utilisateur
        json_part = {}
        txt = conversation["text"].lower()
        json_part["text"] = txt
        json_part["intent"] = "Book flight"
        # Nous n'utiliserons que ce qu'ont écrit les utilisateurs
        if conversation["author"] == "user":
            for act in conversation["labels"]["acts"]:
                entities = []
                for arg in act["args"]:
                    if arg["key"] in ls_entities:
                        entity = {}
                        key = arg["key"].lower()
                        if "val" in arg.keys():
                            val = arg["val"].lower()
                            if val != "-1":
                                startCharIndex = txt.index(val)
                                endCharIndex = startCharIndex + len(val)
                                entity["entity"] = key
                                entity["startPos"] = startCharIndex
                                entity["endPos"] = endCharIndex
                                entities.append(entity)
                json_part["entities"] = entities
                
        if (len(json_part)>0):
            if "entities" in json_part.keys():
                if len(json_part["entities"])>0:
                    luis_data.append(json_part)
    return luis_data

def convert_data(data, ls_entities):
    luis_data = []
    for i in range(data.shape[0]):
        json_part = get_turn_entities(data, i, ls_entities)
        if len(json_part)>0:
            for j in range(len(json_part)):
                luis_data.append(json_part[j])
    return luis_data

In [127]:
# pour convertir les data pour LUIS :
luis_json = convert_data(data[:17], ls_entities) #18 is causing trouble when importing JSON: BadArgument: Invalid value dst_city = "mexico city"

with open('dataset/luis_json.json', 'w') as file:
    json.dump(luis_json, file)
len(luis_json)


11

In [128]:
with open('dataset/FlightBooking_template.json', 'r') as file:
    v0_FlightBooking = json.load(file)

v0_FlightBooking['utterances'] = luis_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified4.json', 'w') as file:
    json.dump(v0_FlightBooking, file, indent=1)

In [124]:
luis_json

[{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
  'intent': 'Book flight',
  'entities': [{'entity': 'dst_city', 'startPos': 27, 'endPos': 35},
   {'entity': 'or_city', 'startPos': 41, 'endPos': 48},
   {'entity': 'str_date', 'startPos': 52, 'endPos': 77},
   {'entity': 'budget', 'startPos': 117, 'endPos': 121}]},
 {'text': "i'm looking for a trip to gotham city leaving from kakariko village on saturday, august 13, 2016. 3 adults for no more than $2400 usd.",
  'intent': 'Book flight',
  'entities': [{'entity': 'dst_city', 'startPos': 26, 'endPos': 37},
   {'entity': 'or_city', 'startPos': 51, 'endPos': 67},
   {'entity': 'str_date', 'startPos': 71, 'endPos': 96},
   {'entity': 'budget', 'startPos': 124, 'endPos': 133}]},
 {'text': 'hello, i would like to book a 2-week trip leaving from melbourne on august 27. i would like to go to mannheim.',
  'intent': 'Book flight',
  'entities': [{'entity': 'or